In [1]:
import pandas as pd
from sklearn.calibration import cross_val_predict
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

<h1>Cross Validate</h1>

<h2>LR - Frequency</h2>

In [5]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador
clf_polarity = LogisticRegression(max_iter=10000)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.44572551 0.46789743 0.47366973 0.47588564 0.48445446]
F1-score Promedio: 0.46952655719986014


In [7]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador
clf_polarity = LogisticRegression(max_iter=10000)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Definir target_names
target_names = ['clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5']  # Reemplaza con las clases reales

# Aplicar la validación cruzada y obtener las predicciones y puntuaciones
y_pred_cv = cross_val_predict(clf_polarity, X_train_final, y_train, cv=kf)
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)

# Imprimir el informe de clasificación para cada fold
for i, (train_index, test_index) in enumerate(kf.split(X_train_final)):
    X_train_fold, X_test_fold = X_train_final[train_index], X_train_final[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    clf_polarity.fit(X_train_fold, y_train_fold)
    y_pred_fold = clf_polarity.predict(X_test_fold)
    
    print(f"\nFold {i + 1}")
    print(classification_report(y_test_fold, y_pred_fold, target_names=target_names))

print("F1-score Promedio:", cv_scores.mean())



Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.44572551 0.46789743 0.47366973 0.47588564 0.48445446]

Fold 1
              precision    recall  f1-score   support

     clase_1       0.44      0.30      0.36        77
     clase_2       0.29      0.22      0.25       105
     clase_3       0.37      0.38      0.37       326
     clase_4       0.43      0.36      0.39       988
     clase_5       0.83      0.88      0.86      3338

    accuracy                           0.72      4834
   macro avg       0.47      0.43      0.45      4834
weighted avg       0.70      0.72      0.71      4834


Fold 2
              precision    recall  f1-score   support

     clase_1       0.48      0.33      0.39        99
     clase_2       0.30      0.27      0.28       116
     clase_3       0.42      0.37      0.39       353
     clase_4       0.45      0.37 

In [8]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer(binary=True)
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador
clf_polarity = LogisticRegression(max_iter=10000)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.45596859 0.44256214 0.46658067 0.46126232 0.48275254]
F1-score Promedio: 0.46182525389098145


<h3>FINAL-MEJOR RESULTADO (Logistic Regression) </h3>

In [3]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer()

X_train_vect = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vect = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

X_train = hstack([X_train_vect, X_train[numeric_features].values])
X_test = hstack([X_test_vect, X_test[numeric_features].values])

clf_lr = LogisticRegression(max_iter=10000)
clf_lr.fit(X_train, y_train)
 
y_pred = clf_lr.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(conf_matrix)
print(classification_rep)


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

[[  54   16   14    8   12]
 [  21   37   43   23   21]
 [  18   34  170  111   89]
 [   6   16  101  430  610]
 [   0    4   48  411 3746]]
              precision    recall  f1-score   support

           1       0.55      0.52      0.53       104
           2       0.35      0.26      0.29       145
           3       0.45      0.40      0.43       422
           4       0.44      0.37      0.40      1163
           5       0.84      0.89      0.86      4209

    accuracy                           0.73      6043
   macro avg       0.52      0.49      0.50      6043
weighted avg       0.72      0.73      0.72      6043



<h2>SVM</h2>

In [2]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer(binary=True)
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador SVM
clf_polarity = SVC()

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.36415662 0.37049847 0.36301339 0.34511615 0.37495746]
F1-score Promedio: 0.3635484186741677


In [ ]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer(binary=True)
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador SVM
clf_polarity = SVC(kernel='linear')

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas



In [ ]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = CountVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador SVM
clf_polarity = SVC(kernel='linear')

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas



In [6]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = TfidfVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador SVM
clf_polarity = SVC(kernel='linear')

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.43765601 0.45100845 0.45128141 0.449842   0.47543115]
F1-score Promedio: 0.45304380522340065


<h2>MLP</h2>

In [3]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = TfidfVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador 
clf_polarity = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=200)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.42233242 0.45010531 0.45362425 0.44193233 0.45075667]
F1-score Promedio: 0.44375019398667853


In [2]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = TfidfVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador 
clf_polarity = MLPClassifier(hidden_layer_sizes=(50, 100), max_iter=400)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.42648638 0.46602788 0.43019199 0.4485405  0.46325729]
F1-score Promedio: 0.4469008060696386


In [2]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = TfidfVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador 
clf_polarity = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=100)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.44340992 0.46363151 0.46515402 0.45158974 0.45462694]
F1-score Promedio: 0.45568242622315625


In [3]:
df_normalizacion = pd.read_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

features = ['Title_Opinion', 'acumuladopositivo', 'acumuladonegative']
numeric_features = ['acumuladopositivo', 'acumuladonegative']

X = df_normalizacion[features]
y = df_normalizacion['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f'\nTamaño de X_train: {len(X_train)} documentos')
print(f'Tamaño de X_test: {len(X_test)} documentos')
print(f'Tamaño de y_train: {len(y_train)} etiquetas')
print(f'Tamaño de y_test: {len(y_test)} etiquetas\n')

# Vectorización de texto
vectorizer_title_opinion = TfidfVectorizer()
X_train_vectorizer = vectorizer_title_opinion.fit_transform(X_train['Title_Opinion'])
X_test_vectorizer = vectorizer_title_opinion.transform(X_test['Title_Opinion'])

# Combinar características numéricas y vectorizadas
X_train_final = hstack([X_train_vectorizer, csr_matrix(X_train[numeric_features].values)])
X_test_final = hstack([X_test_vectorizer, csr_matrix(X_test[numeric_features].values)])

# Inicializar el clasificador 
clf_polarity = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400)

# Inicializar KFold con el número deseado de divisiones (folds)
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Definir la métrica F1-score para la validación cruzada
scoring_metric = make_scorer(f1_score, average='macro')

# Aplicar la validación cruzada y obtener las puntuaciones
cv_scores = cross_val_score(clf_polarity, X_train_final, y_train, cv=kf, scoring=scoring_metric)

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada (F1-score):", cv_scores)
print("F1-score Promedio:", cv_scores.mean())


Tamaño de X_train: 24169 documentos
Tamaño de X_test: 6043 documentos
Tamaño de y_train: 24169 etiquetas
Tamaño de y_test: 6043 etiquetas

Puntuaciones de Validación Cruzada (F1-score): [0.40905099 0.45709282 0.45415696 0.45537018 0.47083514]
F1-score Promedio: 0.44930121691619307
